# Open Coding Society – Github Analytics & Grade Predictor

We expanded the existing OCS dashboard into a richer, more interactive analytics hub backed by secure, JWT-protected APIs and a lightweight “smart” grade model.

## Purpose
- Existing GitHub analytics were sometimes inaccurate, hard to navigate, and couldn’t drill down into specific contributions.  
- Grade estimates were fixed, ignoring the breadth of a student’s work.

## What’s New

### Interactive Dashboard
- Tabbed sections for:
  - **GitHub Analytics**
  - **Smart Grade Predictor**
  - **Admin Search Pane**
  - **Help**
- Clickable **commit cards** linking directly to the GitHub commit
- **Admin tools** to look up users by UID
- Grade Predictor tool based on commits or holeistic view

### API Enhancements (`/api/analytics`)
- All endpoints are **JWT-secured**
- Powered by the `GitHubUser` class using the **GitHub GraphQL API**
- Supports optional **date ranges**, defaulting to trimester 3
- User Endpoints:
  - `GET /github/user`
  - `GET /github/user/commits`
  - `GET /github/user/prs`
  - `GET /github/user/issues`
  - `GET /github/user/issue_comments`
  - `GET /github/user/received_issue_comments`
- Admin Endpoints:
  - `GET /commits/<uid>`
  - `GET /issues/<uid>`

## Smart Grade Predictor
- Two modes:
  - **Basic** – Based on commit count
  - **Smart** – Weighted GitHub activity metrics

## Outcome
Students and instructors now get clearer, drill-down insight into coding activity and a dynamic grade estimate grounded in holistic GitHub metrics — all while maintaining secure access controls.


# My Contributions:
As backend engineer, I tested all existing endpoints for the analytics api in postman and through frontend, switched admin endpoints to get ratehr than post for consistency.
<img src="{{site.baseurl}}/images/final1.png"/>

Added line changes per commit, and total line changes into get_commit_stats function in model file
<img src="{{site.baseurl}}/images/final2.png"/>

Added admin feature on frontend that calls admin-user commits endpoint, allows admin to look at anyone in the database's commits in carded format:

Contributed to integration onto dashboard of OCS, merging with other grade anayltics groups.

# InterTravel
### Group’s purpose
We wanted to create a travel planner with a varities of functions for the user: “To make a budget-friendly travel planner that will allow users to find activities, hotels, restaurants, and much more.”

This project was depolyed and had working log in systems, as well as a backend site.

It would include info for 5 different cities (planned):
- Paris (only one integrated)
- Hong Kong
- New York
- Mumbai
- London

It would include these functionalities:
- Activity Planner: users can search for activities in the city they are visiting and ask questions to a chatbot for help.
- Lodging Listings: users can search for hotels in their city and give them a star rating.
- Packing Portal: users can select which items to carry with them while travelling.
- Cuisine Chronicles: users can view the menu of restaurants and give reviews.
- Fair Fares: users can search for flights to and from their destination and compare prices.
- Budget Brilliance: users can set their budget for the trip and see suggestions.
- Wellness waypoints: users can search for hospitals in their area in the case of an emergency.

### My contributions
I built a section of the website that helps the user plan their travel budget with functional and user-friendly frontend and dynamically updating APIs that connect to the backend. It will include these planned features.
- Budget Plan Assister:
  - takes in user inputed location, budget and any hard set attractions or travel items with predetermined price, user can then also input what they need help with finding (ex. hotels)
  - Outputs budget entries into a nice table with clear depictions, and also the remaining budget
  - integrates smoothly into the overall daily planning system. The user should be able to add budgeting concerns and details to their daily itinerary and travel plan
  - currency converter API

<img src="{{site.baseurl}}/images/final3.png"/>
<img src="{{site.baseurl}}/images/tri2final/remainingbudget.png"/>

## Slient Video Demo
<iframe width="560" height="315" src="https://www.youtube.com/embed/qWp4kdXQ5iU" frameborder="0" allowfullscreen></iframe>

[Or watch the video here!](https://www.youtube.com/watch?v=qWp4kdXQ5iU)

## Dynamic Budgeting API with CRUD and backend database

```py
import jwt
from flask import Blueprint, request, jsonify, g
from flask_restful import Api, Resource
from flask_cors import CORS, cross_origin
from datetime import datetime
from __init__ import app, db  # Ensure db is imported
from api.jwt_authorize import token_required
from model.budgeting import Budgeting  # Assuming your Budgeting model is in the 'budgeting' module
from model.user import User

budgeting_api = Blueprint('budgeting_api', __name__, url_prefix='/api')
CORS(budgeting_api, supports_credentials=True, methods=["GET", "POST", "PUT", "DELETE"])
api = Api(budgeting_api)

class BudgetingAPI:
    """
    Define the API CRUD endpoints for the Budgeting model.
    Operations include creating, retrieving, updating, and deleting budgeting entries.
    """
    
    class _CRUD(Resource):
        @token_required()
        @cross_origin(supports_credentials=True)
        def post(self):
            current_user = g.current_user
            data = request.get_json()
            
            if not data or 'expense' not in data or 'cost' not in data or 'category' not in data:
                return jsonify({"message": "Expense, cost, and category are required"}), 400
            
            budgeting = Budgeting(
                expense=data.get('expense'),
                cost=data.get('cost'),
                category=data.get('category'),
                user_id=current_user.id
            )
            
            db.session.add(budgeting)
            db.session.commit()
            
            return jsonify({"message": "Budgeting entry created successfully"})
        
        @token_required()
        def get(self):
            current_user = g.current_user
            budgeting_entries = Budgeting.query.filter_by(user_id=current_user.id).all()
            return jsonify([entry.read() for entry in budgeting_entries])
        
        @token_required()
        def put(self):
            current_user = g.current_user
            data = request.get_json()
            budgeting_id = data.get('id')
            
            if not budgeting_id:
                return jsonify({"message": "ID is required for updating a budgeting entry"}), 400
            
            budgeting = Budgeting.query.filter_by(id=budgeting_id, user_id=current_user.id).first()
            if not budgeting:
                return jsonify({"message": "Budgeting entry not found"}), 404
            
            budgeting.expense = data.get('expense', budgeting.expense)
            budgeting.cost = data.get('cost', budgeting.cost)
            budgeting.category = data.get('category', budgeting.category)
            
            db.session.commit()
            return jsonify({"message": "Budgeting entry updated successfully"})
        
        @token_required()
        def delete(self):
            current_user = g.current_user
            data = request.get_json()
            budgeting_id = data.get('id')
            
            if not budgeting_id:
                return jsonify({"message": "ID is required for deleting a budgeting entry"}), 400
            
            budgeting = Budgeting.query.filter_by(id=budgeting_id, user_id=current_user.id).first()
            if not budgeting:
                return jsonify({"message": "Budgeting entry not found"}), 404
            
            db.session.delete(budgeting)
            db.session.commit()
            return jsonify({"message": "Budgeting entry deleted successfully"})

api.add_resource(BudgetingAPI._CRUD, '/budgeting')
```
```js
    async function submitBudgeting(expense, cost, category) {
        try {
            const response = await fetch(`${pythonURI}/api/budgeting`, {
                ...fetchOptions,
                method: 'POST',
                headers: { 'Content-Type': 'application/json' },
                body: JSON.stringify({ expense, cost, category, user_id: USER_ID }),
            });

            if (response.ok) {
                createBudgetingTable();
                updateRemainingBudget();
            } else {
                console.error('Failed to submit budgeting entry:', await response.json());
            }
        } catch (error) {
            console.error("Error creating new budgeting entry:", error);
        }
    }
```

## Backend Admin role: customized backend UI to reflect our project with admin features
<img src="{{site.baseurl}}/images/tri2final/backendwebsite.png"/>
```
<div class="col-4">
    <div class="card">
        <img class="card-img-top" src="{{ url_for('static', filename='assets/airplane.jpg') }}" alt="Python Development" height="350">
        <div class="card-body">
        <h5 class="card-title">What are the features of <mark>InterTravel?</mark>?</h5>
        <p class="card-text">
            <ol>
                <li>Wellness Waypoints</li>
                <li>Lodging Listings</li>
                <li>Activity Planner</li>
                <li>Packing Portal</li>
                <li>Cuisine Chronicles</li>
                <li>Fair Fares</li>
                <li>Budget Brilliance</li>
            </ol>
        </p>
        <a class="btn btn-primary" href="https://kiruthic-selvakumar.github.io/travel_frontend/">Check them out!</a>
        </div>
    </div>
</div>
```
```
  async function fetchBudgetingEntries() {
    try {
      const response = await fetch('/api/budgeting');
      if (!response.ok) throw new Error('Failed to fetch budgeting entries');
      const budgetingEntries = await response.json();
      const budgetBody = document.getElementById('budgetBody');
      budgetBody.innerHTML = '';
      budgetingEntries.forEach(entry => {
        budgetBody.innerHTML += `
          <tr data-id="${entry.id}" 
              data-expense="${entry.expense}" 
              data-cost="${entry.cost}" 
              data-category="${entry.category}">
            <td>${entry.id}</td>
            <td>${entry.expense}</td>
            <td>${entry.cost}</td>
            <td>${entry.category}</td>
            <td>
              <button class="btn btn-primary edit-btn" data-id="${entry.id}">Edit</button>
              <button class="btn btn-danger delete-btn" data-id="${entry.id}">Delete</button>
            </td>
          </tr>`;
      });
    } catch (error) {
      console.error('Error fetching budgeting entries:', error);
    }
  }
```

# Unique Technical Skills Summary

Through two major full-stack projects — *Open Coding Society (OCS) Analytics Hub* and *InterTravel Budget Planner* — I developed strong backend, frontend, data science, and collaboration skills.

### Project Planning & Management
- Organized team tasks using issues, user stories, flowcharts, and kanbans, set goals, and kept the project on schedule.

### Backend Engineering & API Development
- Built and tested JWT-secured REST APIs with Flask.
- Used GitHub GraphQL API for detailed user data.
- Created CRUD endpoints for budgeting using SQLAlchemy.
- Ensured access control for user/admin roles.

### Full-Stack Integration & UX
- Connected backend APIs to interactive frontends
- Made clickable commit cards and admin search tools, Designed editable budgeting tables with real-time updates.

### Project Collaboration with Agile Methodology
- Used agile practices such as sprints, stand-ups, and iterative development to ensure continuous progress and adaptability.
- Helped merge code across teams, worked on branches and used PRs, tested with Postman, debugged with console.

In conclusion, these projects helped me demonstrate my full-stack technical skills, attention to detail, and teamwork. I’m proud of the impact I made and eager to grow further with new challenges ahead.